# Hackathon Dotz

##### Classificar as observações por (SUB-CATEGORIA | CATEGORIA | DEPARTAMENTO)

In [0]:
# Importação dos pacotes para Analise
 
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sct
import statsmodels.api as sm
from google.colab import files  
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [36]:
#Carregar os dados para o google colab

uploaded = files.upload()

In [0]:
# Importação dos dataset

df = pd.read_csv('Hackathon_Base_Treino_comdep.csv')
df2 = pd.read_csv('Hackathon_Base_Teste.csv')

### Etapa de Preparação do Dados

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)
# Dados de Teste
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
pontuacao=['.',',','-','+',':',';','&','+','/','!','#','%','(',')','  ']
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')
  # Dados de Teste
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, ' ')

In [39]:
df.head(2)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE,PASTA INT VITAPOWER KG AMEND SHOT
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C


In [0]:
# Variáveis importantes
# Palavras para retirar da análise
stop_words = ['de','do','da','kg','g','l','ml','e','gfa','un','lt','n','la','daiso',
              'acasa','massa','seara','friboi','big','granel','peca','montana',
              'temperado','chef','real','bananinha','bd','ceratti','sulita','macedo',
              'copacol','sadia','daucy','angus','pct','samurai','mezzani','fr','basilar',
              'colonial','fr','cesabela','nonna','rana','brasa','burguer','burguers',
              'perdigao','sodebo','atelie','mania','beef','germania','miguel','regency',
              'corttex','pompets','oceane','vermelho','azul','branco','verde','amarelo',
              'dona','clara','belem','sao','domingos','jorge','umbra','prisma','imp',
              'juta','lupo','urban','sport']
# Tipo acentuação
strip_accents = 'unicode'
# Tamanho da validação de teste
test_size = 0.2
# Interações no modelo
max_iter = 2000
# Tipo do vetor
tipo_vetor = HashingVectorizer(stop_words=stop_words, strip_accents=strip_accents)

### Etapa 1 - Classificando DEPARTAMENTO

In [0]:
#Criando a nova para classificar DEPARTAMENTO
base = df
 
# Selecionando apenas o item a ser classificado e o target do DF principal
base_data = base['DESC_AJUSTADA']
base_target = base[['CATEGORIA','SUB-CATEGORIA']]
 
# Transformando a descrição em vetor
vetor = tipo_vetor
vetor_x = vetor.fit_transform(base_data)
 
# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(vetor_x, base_target, test_size=test_size, random_state=27)

In [42]:
# Mostrar o tempo de processamento
%%time

# Aqui nós configuramos o modelo para treinar o classificador
clf = MultiOutputClassifier(KNeighborsClassifier(n_neighbors=12))

# Executar o treinamento
clf.fit(x_train, y_train)

# Avaliando a performance com predição
predicted = clf.predict(x_test)
 
print('\n###Indicadores Classificação###\n')
print('Score: ', clf.score(x_test, y_test), '\n')


###Indicadores Classificação###

Score:  0.5792821444797819 

CPU times: user 5.27 s, sys: 23.9 ms, total: 5.3 s
Wall time: 5.3 s


In [43]:
# Predição em novos dados com o DF  de teste
base_novos = df2['DESC_AJUSTADA']

# Tranformando descrição em vetor
vetor_n = vetor.transform(base_novos)

#Previsão dos novos dados
predicted_novos = clf.predict(vetor_n)

df_dotz = pd.DataFrame(data=predicted_novos, columns=['CATEGORIA','SUB-CATEGORIA'])
df_dotz['DESCRIÇÃO PARCEIRO'] = df2['DESCRIÇÃO PARCEIRO']
df_dotz

,CATEGORIA,SUB-CATEGORIA,DESCRIÇÃO PARCEIRO
0,SAL,SAL DO HIMALAIA,SAL ROSA HIMALAIA C/ ALHO 500G
1,ENFEITES DE NATAL,ENFEITES DE ÁRVORE DE NATAL,JG BOLA NATAL ACASA C/17 DR/PR/BC R952
2,CHÁ,FRUTAS,MIST PREP LIQ CHA VDE LINEA 56ML-CX LIMAO
3,ACESSÓRIOS MÃO E PÉ,ALICATE,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO
4,PIZZA,CALABRESA,MEIA PIZZA PEPPERONI UN
...,...,...,...
9428,BISCOITO,IMPORTADO,BISC IMP ITAL GRISBI DOCE CAPPUCCINO CX 150G
9429,VINHO,VINHO ARGENTINO,VINHO ARGENTINO SANTA ANA BCO SECO 700ML
9430,ALGODÃO,BOLA,ALGODAO BOLA ISABABY 50G .
9431,ORGANIZADOR,CAIXA ORGANIZADORA,ORG PIA OU DISC OP590TQ
